# Base Architecture Below

# English-to-Spanish translation with a sequence-to-sequence Transformer

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2021/05/26<br>
**Last modified:** 2021/05/26<br>
**Description:** Implementing a sequence-to-sequene Transformer and training it on a machine translation task.

## We modify the above base model by applying our dataset, augmentation methods, and training procedure.
### The filename indicates what changes are being measured/experimented here. Note that not all files are here as some files are still being recovered.

**Authors:** Chaitanya Ravuri, Karissa Sanchez, Chanwoo Yoon

## Introduction

In this example, we'll build a sequence-to-sequence Transformer model, which
we'll train on an English-to-Spanish machine translation task.

You'll learn how to:

- Vectorize text using the Keras `TextVectorization` layer.
- Implement a `TransformerEncoder` layer, a `TransformerDecoder` layer,
and a `PositionalEmbedding` layer.
- Prepare data for training a sequence-to-sequence model.
- Use the trained model to generate translations of never-seen-before
input sentences (sequence-to-sequence inference).

The code featured here is adapted from the book
[Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition)
(chapter 11: Deep learning for text).
The present example is fairly barebones, so for detailed explanations of
how each building block works, as well as the theory behind Transformers,
I recommend reading the book.

## Setup

In [1]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 4.3 MB/s 


In [1]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

## Downloading the data

We'll be working with an English-to-Spanish translation dataset
provided by [Anki](https://www.manythings.org/anki/). Let's download it:

In [2]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

2638744/2638744 [==============================] - 0s 0us/step


## Parsing the data

Each line contains an English sentence and its corresponding Spanish sentence.
The English sentence is the *source sequence* and Spanish one is the *target sequence*.
We prepend the token `"[start]"` and we append the token `"[end]"` to the Spanish sentence.

In [3]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    eng = "[start] " + eng + " [end]"
    text_pairs.append((spa, eng))

Here's what our sentence pairs look like:

In [4]:
for _ in range(5):
    print(random.choice(text_pairs))

('Me quiero lavar.', '[start] I want to wash myself. [end]')
('Él sonríe cada vez que le veo.', '[start] He smiles every time I see him. [end]')
('La mayoría de los accidentes ocurren cerca de casa.', '[start] Most accidents happen in the neighborhood. [end]')
('Ningún estudiante fue ahí ayer.', '[start] No students went there yesterday. [end]')
('¿Y si algo se rompe?', '[start] What if something gets broken? [end]')


Now, let's split the sentence pairs into a training set, a validation set,
and a test set.

In [5]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


## Vectorizing the text data

We'll use two instances of the `TextVectorization` layer to vectorize the text
data (one for English and one for Spanish),
that is to say, to turn the original strings into integer sequences
where each integer represents the index of a word in a vocabulary.

The English layer will use the default string standardization (strip punctuation characters)
and splitting scheme (split on whitespace), while
the Spanish layer will use a custom standardization, where we add the character
`"¿"` to the set of punctuation characters to be stripped.

Note: in a production-grade machine translation model, I would not recommend
stripping the punctuation characters in either language. Instead, I would recommend turning
each punctuation character into its own token,
which you could achieve by providing a custom `split` function to the `TextVectorization` layer.

In [60]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the vectorized source sentence and `encoder_inputs` is the target sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

In [61]:

def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 20 steps long):

In [62]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


## Making the low-resource datasets

For our low resource datasets, we take 10% of the high resource dataset. The dataset is already shuffled, so we can just take the first 10%.

In [63]:
frac_low_resource = 0.1

train_pairs_low = train_pairs[:int(frac_low_resource * len(train_pairs))]
val_pairs_low = val_pairs[:int(frac_low_resource * len(val_pairs))]

train_eng_texts_low = [pair[0] for pair in train_pairs_low]
train_spa_texts_low = [pair[1] for pair in train_pairs_low]
eng_vectorization.adapt(train_eng_texts_low)
spa_vectorization.adapt(train_spa_texts_low)

train_ds_low = make_dataset(train_pairs_low)
val_ds_low = make_dataset(val_pairs_low)

print(f"{len(train_pairs_low)} low resource training pairs")
print(f"{len(val_pairs_low)} low resource validation pairs")

8327 low resource training pairs
1784 low resource validation pairs


In [ ]:
print(len(train_pairs))
print(len(train_pairs_low))
print(frac_low_resource * len(train_pairs))

num = 0
for input, target in train_ds_low:
    print('----------')
    print(input)

    print('--')

    print(target)
    print('----------')
    if num > 5:
        break
    num += 1

## Building the model

Our sequence-to-sequence Transformer consists of a `TransformerEncoder`
and a `TransformerDecoder` chained together. To make the model aware of word order,
we also use a `PositionalEmbedding` layer.

The source sequence will be pass to the `TransformerEncoder`,
which will produce a new representation of it.
This new representation will then be passed
to the `TransformerDecoder`, together with the target sequence so far (target words 0 to N).
The `TransformerDecoder` will then seek to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking
(see method `get_causal_attention_mask()` on the `TransformerDecoder`).
The `TransformerDecoder` sees the entire sequences at once, and thus we must make
sure that it only uses information from target tokens 0 to N when predicting token N+1
(otherwise, it could use information from the future, which would
result in a model that cannot be used at inference time).

In [64]:

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        # print('decode input:', inputs)
        # print('mask', mask)

        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        # print('causal:', causal_mask)
        # print('padding:', padding_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis] # diagnonal array with 1's and 0's
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1])) # turning it into a 3d matrix
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config


Next, we assemble the end-to-end model.

In [65]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_input")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

# encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
# x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
# encoder = keras.Model(encoder_inputs, encoder_outputs)

# decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
# encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
# x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
# x = layers.Dropout(0.5)(x)
# decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
# decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

# decoder_outputs = decoder([decoder_inputs, encoder_outputs])
# transformer = keras.Model(
#     [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
# )

In [66]:
# encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
# x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
# encoder = keras.Model(encoder_inputs, encoder_outputs)

# decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
# encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
# x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
# x = layers.Dropout(0.5)(x)
# decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
# decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

# decoder_outputs = decoder([decoder_inputs, encoder_outputs])
# transformer = keras.Model(
#     [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"

embed_dim = 256
latent_dim = 2048
num_heads = 8

# encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

# decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
# transformer
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)
# model settings
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy()

train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

## Training our model

We'll use accuracy as a quick way to monitor training progress on the validation data.
Note that machine translation typically uses BLEU scores as well as other metrics, rather than accuracy.

Here we only train for 1 epoch, but to get the model to actually converge
you should train for at least 30 epochs.

In [59]:
# import time

# start_time = time.time()

# epochs = 30
# sentence_length = 20 # data set comes in this length
# spa_vocab = spa_vectorization.get_vocabulary()
# spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
# # training loop
# for epoch in range(epochs):
#     print('EPOCH:', epoch)
#     # for each batch
#     for step, (x_batch_train, y_batch_train) in enumerate(train_ds_low):
#         # have gradient tape to keep track of gradients
#         with tf.GradientTape() as tape:
#             # since this has no teacher forcing, we have to sequentially pass in previously generated tokens
#             decoded_sentence = "[start]" # starting with our start token
#             enc_input = x_batch_train["encoder_inputs"] 
#             enc_output = encoder(enc_input) # get the encoder output
#             for i in range(sentence_length): # for 20 words in a seq
#                 tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1] # decoded sentence to tokens
#                 print(tokenized_target_sentence)
#                 # print(enc_input.shape)
#                 # print(tokenized_target_sentence.shape)
#                 predictions = transformer([enc_input, tokenized_target_sentence]) # pass the current seq and the enc_input to the transformer
            
#                 sampled_token_index = np.argmax(predictions[0, i, :]) # get the prediction
#                 if sampled_token_index not in spa_index_lookup:
#                     sampled_token = '[UNK]'
#                 else:
#                     sampled_token = spa_index_lookup[sampled_token_index]
#                 decoded_sentence += " " + sampled_token # convert it to actual word seq
#             tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1] # prediction
#             # print(y_batch_train)
#             # print(predictions)
#             loss_value = loss_fn(y_batch_train, predictions)
#         # update
#         grads = tape.gradient(loss_value, transformer.trainable_weights)
#         optimizer.apply_gradients(zip(grads, transformer.trainable_weights))
#         train_acc_metric.update_state(y_batch_train, predictions)

#         if step % 200 == 0:
#             print(
#                 "Training loss (for one batch) at step %d: %.4f"
#                 % (step, float(loss_value))
#             )
#             print("Seen so far: %d samples" % ((step + 1) * batch_size))
        
#         # Display metrics at the end of each epoch.
#         train_acc = train_acc_metric.result()
#         print("Training acc over epoch: %.4f" % (float(train_acc),))

#         # Reset training metrics at the end of each epoch
#         train_acc_metric.reset_states()

#         # Run a validation loop at the end of each epoch.
#         for x_batch_val, y_batch_val in val_ds_low:

#             decoded_sentence = "[start]"
#             enc_input = x_batch_train["encoder_inputs"]
#             enc_output = encoder(enc_input)
#             for i in range(sentence_length):
#                 tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
#                 predictions = transformer([enc_input, tokenized_target_sentence])
            
#                 sampled_token_index = np.argmax(predictions[0, i, :])
#                 sampled_token = spa_index_lookup[sampled_token_index]
#                 decoded_sentence += " " + sampled_token
#             tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]

#             # Update val metrics
#             val_acc_metric.update_state(y_batch_val, predictions)
#         val_acc = val_acc_metric.result()
#         val_acc_metric.reset_states()
#         print("Validation acc: %.4f" % (float(val_acc),))
#         print("Time taken: %.2fs" % (time.time() - start_time))
            

# transformer.summary()
# transformer.compile(
#     "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
# )
# transformer.fit(train_ds_low, epochs=epochs, validation_data=val_ds_low)

EPOCH: 0
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=in

KeyboardInterrupt: ignored

In [83]:
arr = tf.constant([[1,2], [3,4]])
print(arr)



arr[:, 0] += tf.constant([5,5])
print(arr)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


TypeError: ignored

In [98]:
import time

start_time = time.time()

epochs = 30
sentence_length = 20 # data set comes in this length
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
# training loop
for epoch in range(epochs):
    print('EPOCH:', epoch)
    # for each batch
    for step, (x_batch_train, y_batch_train) in enumerate(train_ds_low):
        temp_batch_size = x_batch_train["encoder_inputs"].shape[0]
        # have gradient tape to keep track of gradients
        with tf.GradientTape() as tape:
            # since this has no teacher forcing, we have to sequentially pass in previously generated tokens
            decoded_sentence = "[start]" # starting with our start token
            tile_dim = tf.constant([temp_batch_size,1])
            tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
            tokenized_target_sentence = tf.tile(tokenized_target_sentence, tile_dim)
            enc_input = x_batch_train["encoder_inputs"] 
            enc_output = encoder(enc_input)
            for i in range(sentence_length): # for 20 words in a seq
                predictions = transformer([enc_input, tokenized_target_sentence]) # pass the current seq and the enc_input to the transformer
                sampled_token_index = tf.math.argmax(predictions, axis=2)
                sampled_token_indices = sampled_token_index[:, i]
                a = np.zeros((temp_batch_size, sentence_length))
                a[:, i] = sampled_token_indices
                tokenized_target_sentence += a
            loss_value = loss_fn(y_batch_train, predictions)
        # update
        grads = tape.gradient(loss_value, transformer.trainable_weights)
        optimizer.apply_gradients(zip(grads, transformer.trainable_weights))
        train_acc_metric.update_state(y_batch_train, predictions)
        # break
        if step % 10 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * temp_batch_size))
        
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

        # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_ds_low:
        temp_batch_size = x_batch_val["encoder_inputs"].shape[0]
        decoded_sentence = "[start]" # starting with our start token
        tile_dim = tf.constant([temp_batch_size,1])
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        tokenized_target_sentence = tf.tile(tokenized_target_sentence, tile_dim)
        enc_input = x_batch_val["encoder_inputs"] 
        enc_output = encoder(enc_input)

        for i in range(sentence_length):
            predictions = transformer([enc_input, tokenized_target_sentence])
            sampled_token_index = tf.math.argmax(predictions, axis=2)
            sampled_token_indices = sampled_token_index[:, i]
            a = np.zeros((temp_batch_size, sentence_length))
            a[:, i] = sampled_token_indices
            tokenized_target_sentence += a

        val_acc_metric.update_state(y_batch_val, predictions)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))
    # break

EPOCH: 0
Training loss (for one batch) at step 0: 2.6222
Seen so far: 64 samples
Training loss (for one batch) at step 10: 2.4517
Seen so far: 704 samples
Training loss (for one batch) at step 20: 2.3709
Seen so far: 1344 samples
Training loss (for one batch) at step 30: 2.8070
Seen so far: 1984 samples
Training loss (for one batch) at step 40: 2.4120
Seen so far: 2624 samples


KeyboardInterrupt: ignored

## Decoding test sentences

Finally, let's demonstrate how to translate brand new English sentences.
We simply feed into the model the vectorized English sentence
as well as the target token `"[start]"`, then we repeatedly generated the next token, until
we hit the token `"[end]"`.

In [ ]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def lookup_words(x, vocab):
  return [vocab[i] for i in x]

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    output = []
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, '\t', translated)

Probablemente él terminará el trabajo para mañana. 	 [start] probably probably it for the job tomorrow [end]
Tom lavó mucha ropa hoy. 	 [start] tom did very do today [end]
Ella es una de las mejores bailarinas del mundo. 	 [start] she is a left of the best of the world [end]
Estuve pensando en ti el día de hoy. 	 [start] i was at you thinking of me today [end]
Mi padre se dedica al comercio exterior. 	 [start] my father became him to the speak [end]
Detesto ser una molestia. 	 [start] i hate to be a mary [end]
Tomás quiere ser útil. 	 [start] tom wants to be try [end]
Los detalles del plan fueron revelados. 	 [start] details will be out of the plan [end]
Dile a Tom que voy. 	 [start] tell tom [end]
En este momento están descansando. 	 [start] in this time is it [end]
Guárdalo. 	 [start] keep it [end]
Ustedes no deberían haberlo dicho. 	 [start] you shouldnt have too much you [end]
De vez en cuando ella juega tenis. 	 [start] lets play in when she is tennis [end]
Está la posibilidad de 

After 30 epochs, we get results such as:

> She handed him the money.
> [start] ella le pasó el dinero [end]

> Tom has never heard Mary sing.
> [start] tom nunca ha oído cantar a mary [end]

> Perhaps she will come tomorrow.
> [start] tal vez ella vendrá mañana [end]

> I love to write.
> [start] me encanta escribir [end]

> His French is improving little by little.
> [start] su francés va a [UNK] sólo un poco [end]

> My hotel told me to call you.
> [start] mi hotel me dijo que te [UNK] [end]

In [ ]:
import sacrebleu

In [ ]:
from sacrebleu.metrics import bleu
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    output = [spa_vectorization(["[start]"])[0, 0]]

    # output = spa_vectorization(["[start]"])[:, :-1]
    print(output)

    # decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        # tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, output])

        sampled_token_index = np.argmax(predictions[0, i, :])
        output[i+1] = sampled_token_index
        # print(sampled_token_index)

        # sampled_token = spa_index_lookup[sampled_token_index]
        # decoded_sentence += " " + sampled_token
        # print(spa_vectorization([decoded_sentence])[:, :-1])
        # break

        if spa_index_lookup[sampled_token_index] == "[end]":
            break
    print(output)
    return output

def compute_BLEU(test_pairs):
  bleu_scores = []
  for i, (x, y) in enumerate(test_pairs):
    out = decode_sequence(x)
    # print(out, '|', y)
    bleu_scores.append(sacrebleu.raw_corpus_bleu([out], [[y]], .01).score)
    break
    # if i % 100 == 0:
    #   print(i, np.mean(bleu_scores))

  return np.mean(bleu_scores)

In [ ]:
print(compute_BLEU(test_pairs[:100]))

[<tf.Tensor: shape=(), dtype=int64, numpy=2>]


InvalidArgumentError: ignored

In [ ]:
transformer.save_weights('./checkpoints/checkpoint1')